A notebook for calculating N_HI upper limits for a non-detection in VLA data

Assuming a distance of ~10 Mpc, use a representative recessional velocity of 700 km/s (but confirm that noise is flat over the range ~200-1500 km/s)

First, need to get an appropriate cube. I generally prefer a robust weighting with VLA for better image fidelity / dirty beam characteristics, but also do a naturally weighted cube for the sensitivity. Use a relatively wide channel of 27.776kHz (~ 5-6 km/s; bin 4) and also 97.216 kHz (~ 20 km/s; bin 14) - control how noise scales. (It definitely scales so that noise goes down with sqrt(nchan). But the NHI limit scales linearly with deltav, which is why I'll have a higher limit for a larger assumed intrinsic velocity width.

I am making my cubes with some light cleaning. This is because there is the other galaxy (Ursa Major cluster) that is showing up, and I want to make sure its dirty beam response isn't impacting things for me.

I need to know the channels over which I want to calculate the noise. For the b14 data, that would be 220-284. For the b4 data, it would be 771-993

I will want to calculate the noise over that channel range, then convert to a column density sensitivity. To do that, I will also need the beam size and the delta v. For the b4 data, I will scale to something that is ~18-20 km/s; for the b14 data, I will use the channel size. I will need to calculate what the channel size is in km/s - do I already have helper code for this? (If not, maybe write it) Shouldn't really matter if it's radio or optical; do radio because it's easier / more internally consistent.

In [3]:
import astropy.units as u
import numpy as np
from astropy import constants as const

In [1]:
#  imstat(imagename='lightclean_b14_nat.image',chans='220~284')   
rms_b14_nat = 0.00061593

# imstat(imagename='lightclean_b14_rob05.image',chans='220~284') 
rms_b14_rob = 0.00066509

# imstat(imagename='lightclean_b4_nat.image',chans='771~993')
rms_b4_nat = 0.00113809

# imstat(imagename='lightclean_b4_rob05.image',chans='771~993')
rms_b4_rob = 0.0012021

In [6]:
delta_nu_b4 = 27.776 * u.kHz
delta_nu_b14 = 97.216 * u.kHz
restfreq = 1420.405752 * u.MHz

In [8]:
# get channel sizes in (radio) velocity
def convert_chan_freq_vel(chan_res):
    freq_to_vel = u.doppler_radio(restfreq)
    offsetfreq = restfreq - chan_res
    vel = restfreq.to(u.km/u.s, equivalencies = freq_to_vel)
    offsetvel = offsetfreq.to(u.km/u.s, equivalencies = freq_to_vel)
    new_res = offsetvel-vel
    return new_res

delta_v_b4 = convert_chan_freq_vel(delta_nu_b4)
delta_v_b14 = convert_chan_freq_vel(delta_nu_b14)

print(delta_v_b4, delta_v_b14)

5.862434238725893 km / s 20.518519835540626 km / s


In [9]:
# get beam sizes from casa
bmaj_nat = 73.4*u.arcsec
bmin_nat = 49.5*u.arcsec
bmaj_rob = 60.6 * u.arcsec
bmin_rob = 40.9 * u.arcsec

In [20]:
# nhi = 1.823e18 * Tb * linewidth
# tb = 606 * S(mJy) / (bmaj*bmin)

nhi_nat_b14 = 1.823e18 * rms_b14_nat * 1e3 * 606 * delta_v_b14 / (bmaj_nat * bmin_nat) * u.arcsec * u.arcsec / u.km * u.s

print(nhi_nat_b14, 3*nhi_nat_b14)

3.842690618859678e+18 1.1528071856579035e+19


In [21]:
nchan_b4_18 = 18 / delta_v_b4 * u.km / u.s

nhi_nat_b4_18kms = 1.823e18 * rms_b4_nat /np.sqrt(nchan_b4_18) * 1e3 * 606  * 18 / (bmaj_nat * bmin_nat) * u.arcsec * u.arcsec 

print(nhi_nat_b4_18kms, 3*nhi_nat_b4_18kms)

3.5547567367366113e+18 1.0664270210209833e+19


In [22]:

nhi_rob_b14 = 1.823e18 * rms_b14_rob * 1e3 * 606 * delta_v_b14 / (bmaj_rob * bmin_rob) * u.arcsec * u.arcsec / u.km * u.s

print(nhi_rob_b14, 3*nhi_rob_b14)

6.082607696228884e+18 1.8247823088686653e+19


In [23]:

nhi_rob_b4_18kms = 1.823e18 * rms_b4_rob / np.sqrt(nchan_b4_18) * 1e3 * 606 * 18/ (bmaj_rob * bmin_rob) * u.arcsec * u.arcsec 

print(nhi_rob_b4_18kms, 3*nhi_rob_b4_18kms)

5.504009849440031e+18 1.6512029548320092e+19


So the ~3-sigma N_HI sensitivity limit for the naturally weighted data, for 18-20 km/s intrinsic width is 1-1.2 x 10^19 atoms cm^-2. For the robust data, it is 1.7-1.8x10^19 atoms cm^-2. In any case, should have easily been able to detect a resolved source, order of magnitude lower than the classic HI disk (1.25x10^20)